In [8]:
from selenium import webdriver
import time as t
import bs4
import pandas as pd

In [38]:
driver = webdriver.Firefox()

In [3]:
data = {'date': [], 'time': [], "title": [], 'content': [], 'co_id': [], 'date_of_event': []}

In [43]:
co_id='6796'
syear=111

In [44]:
#10年資料
for i in range(10):
    year= str(syear-i)
    url="https://mops.twse.com.tw/mops/web/ajax_t05st01?co_id="+co_id+"&year="+year+"&TYPEK=all&encodeURIComponent=1&step=1&firstin=1&off=1"
    driver.get(url)

    t.sleep(3)
    original_window = driver.current_window_handle
    #assert len(driver.window_handles) == 1

    for i in driver.find_elements("xpath","//input[@value='詳細資料']"):
        i.click()
        for window_handle in driver.window_handles:
            t.sleep(3)
            if window_handle != original_window:
                driver.switch_to.window(window_handle)
                break
        soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
        rowDatas=soup.find_all('td')
        
        #錯誤排除
        if(len(rowDatas))<6:
            t.sleep(30)
            driver.close()
            driver.switch_to.window(original_window)
            i.click()
            for window_handle in driver.window_handles:
                t.sleep(3)
                if window_handle != original_window:
                    driver.switch_to.window(window_handle)
                    break
            soup = bs4.BeautifulSoup(driver.page_source, 'html.parser')
            rowDatas=soup.find_all('td')
        #date
        date=(rowDatas[6].text.replace(u'\xa0', u''))
        date=date.replace(u'/',u'')
        y=str(int(date[:3])+1911)
        m=date[3:5]
        d=date[5:7]
        date=y+m+d
        data['date'].append(date)

        #time
        data['time'].append(rowDatas[8].text.replace(u'\xa0', u''))

        #title
        data['title'].append(rowDatas[16].text.replace(u'\xa0', u''))

        #date of event 實際發生日
        date=(rowDatas[20].text.replace(u'\xa0', u''))
        date=date.replace(u'/',u'')
        y=str(int(date[:3])+1911)
        m=date[3:5]
        d=date[5:7]
        date=y+m+d
        data['date_of_event'].append(date)

        #content
        data['content'].append(rowDatas[22].text.replace(u'\xa0', u''))

        #co_id
        data['co_id'].append(co_id)

        driver.close()
        driver.switch_to.window(original_window)

    #driver.quit()
    print(year)

111
110
109
108
107
106
105
104
103
102


In [48]:
#轉成DataFrame
result=pd.DataFrame(data)

In [49]:
#to csv
compression_opts = dict(method='zip',
                        archive_name='material_information.csv')  
result.to_csv('material_information.zip', index=True,
          compression=compression_opts) 